In [1]:
# Initial imports
import pandas as pd
import datetime
from path import Path
# Import our dependencies
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier
import joblib
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
#pd.set_option('display.max_columns', None)

In [2]:
# Load the arrestee.csv dataset.
arestee_df = pd.read_csv("IL/arrestee.csv", parse_dates=['ARREST_DATE'])

# Load the arrestee_types.csv dataset.
arrest_types_df = pd.read_csv("IL/NIBRS_ARREST_TYPE.csv")

# Load the offense_types.csv dataset.
offense_types_df = pd.read_csv("IL/NIBRS_OFFENSE_TYPE.csv")

# Load the offenses.csv dataset.
offenses_df = pd.read_csv("IL/NIBRS_OFFENSE.csv")

# Load the location_type.csv dataset.
location_df = pd.read_csv("IL/NIBRS_LOCATION_TYPE.csv")
location_df.drop('LOCATION_CODE', axis=1, inplace=True)

In [139]:
# Join arestee_df and arrest_types_df on ARREST TYPE ID
joined_df = arestee_df.merge(arrest_types_df, on ="ARREST_TYPE_ID")
joined_df

,DATA_YEAR,ARRESTEE_ID,INCIDENT_ID,ARRESTEE_SEQ_NUM,ARREST_DATE,SEASON,ARREST_TYPE_ID,MULTIPLE_INDICATOR,OFFENSE_TYPE_ID,AGE_ID,AGE_NUM,SEX_CODE,RACE_ID,ETHNICITY_ID,RESIDENT_CODE,UNDER_18_DISPOSITION_CODE,CLEARANCE_IND,AGE_RANGE_LOW_NUM,AGE_RANGE_HIGH_NUM,ARREST_TYPE_CODE,ARREST_TYPE_NAME
0,2019,33189213,111310423,1,2019-03-01,winter,2,N,16,5,13,F,2,2,R,R,NaN,13,0,S,Summoned / Cited
1,2019,33182447,111310447,1,2019-03-05,winter,2,N,16,5,29,F,1,2,R,NaN,NaN,29,0,S,Summoned / Cited
2,2019,33189271,111310549,2,2019-03-18,winter,2,N,16,5,30,M,2,2,R,NaN,NaN,30,0,S,Summoned / Cited
3,2019,33182336,111310215,1,2019-02-12,winter,2,N,23,5,32,F,1,2,R,NaN,NaN,32,0,S,Summoned / Cited
4,2019,31971399,107364860,1,2019-01-21,winter,2,N,23,5,35,F,1,2,N,NaN,NaN,35,0,S,Summoned / Cited
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4135,2019,36247339,118888164,1,2019-10-16,autumn,3,N,51,5,55,F,2,2,R,NaN,NaN,55,0,T,Taken INTO Custody
4136,2019,36255107,118879914,1,2019-10-08,autumn,3,N,48,5,39,M,2,2,N,NaN,NaN,39,0,T,Taken INTO Custody
4137,2019,36255122,118879927,1,2019-10-08,autumn,3,N,27,5,50,M,2,2,R,NaN,NaN,50,0,T,Taken INTO Custody
4138,2019,36247354,118879933,1,2019-05-21,spring,3,N,27,5,29,M,2,2,R,NaN,NaN,29,0,T,Taken INTO Custody


In [140]:
# Join joined_df and offense_types on OFFENSE_TYPE_ID.
joined_df2 = joined_df.merge(offense_types_df, on ="OFFENSE_TYPE_ID")
joined_df2

,DATA_YEAR,ARRESTEE_ID,INCIDENT_ID,ARRESTEE_SEQ_NUM,ARREST_DATE,SEASON,ARREST_TYPE_ID,MULTIPLE_INDICATOR,OFFENSE_TYPE_ID,AGE_ID,AGE_NUM,SEX_CODE,RACE_ID,ETHNICITY_ID,RESIDENT_CODE,UNDER_18_DISPOSITION_CODE,CLEARANCE_IND,AGE_RANGE_LOW_NUM,AGE_RANGE_HIGH_NUM,ARREST_TYPE_CODE,ARREST_TYPE_NAME,OFFENSE_CODE,OFFENSE_NAME,CRIME_AGAINST,CT_FLAG,HC_FLAG,HC_CODE,OFFENSE_CATEGORY_NAME,OFFENSE_GROUP
0,2019,33189213,111310423,1,2019-03-01,winter,2,N,16,5,13,F,2,2,R,R,NaN,13,0,S,Summoned / Cited,35A,Drug/Narcotic Violations,Society,N,Y,NaN,Drug/Narcotic Offenses,A
1,2019,33182447,111310447,1,2019-03-05,winter,2,N,16,5,29,F,1,2,R,NaN,NaN,29,0,S,Summoned / Cited,35A,Drug/Narcotic Violations,Society,N,Y,NaN,Drug/Narcotic Offenses,A
2,2019,33189271,111310549,2,2019-03-18,winter,2,N,16,5,30,M,2,2,R,NaN,NaN,30,0,S,Summoned / Cited,35A,Drug/Narcotic Violations,Society,N,Y,NaN,Drug/Narcotic Offenses,A
3,2019,31975212,107368302,2,2019-01-11,winter,2,N,16,5,35,M,1,2,R,NaN,NaN,35,0,S,Summoned / Cited,35A,Drug/Narcotic Violations,Society,N,Y,NaN,Drug/Narcotic Offenses,A
4,2019,31975196,107368239,1,2019-01-06,winter,2,N,16,5,39,F,1,2,R,NaN,NaN,39,0,S,Summoned / Cited,35A,Drug/Narcotic Violations,Society,N,Y,NaN,Drug/Narcotic Offenses,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4135,2019,37098216,121196510,3,2019-08-21,summer,3,N,30,5,64,M,1,2,R,NaN,NaN,64,0,T,Taken INTO Custody,40A,Prostitution,Society,N,Y,NaN,Prostitution Offenses,A
4136,2019,37098227,121196510,23,2019-08-28,summer,3,N,30,5,71,M,1,2,N,NaN,NaN,71,0,T,Taken INTO Custody,40A,Prostitution,Society,N,Y,NaN,Prostitution Offenses,A
4137,2019,37098230,121196510,26,2019-09-02,summer,3,N,30,5,54,M,1,2,N,NaN,NaN,54,0,T,Taken INTO Custody,40A,Prostitution,Society,N,Y,NaN,Prostitution Offenses,A
4138,2019,34055873,113383897,1,2019-01-22,winter,3,N,38,5,25,M,1,1,R,NaN,NaN,25,0,T,Taken INTO Custody,09B,Negligent Manslaughter,Person,N,Y,NaN,Homicide Offenses,A


In [141]:
# Merge joined_df2 and offenses_df on INCIDENT_ID.
joined_df3 = joined_df2.merge(offenses_df, on ="INCIDENT_ID")
joined_df3

,DATA_YEAR_x,ARRESTEE_ID,INCIDENT_ID,ARRESTEE_SEQ_NUM,ARREST_DATE,SEASON,ARREST_TYPE_ID,MULTIPLE_INDICATOR,OFFENSE_TYPE_ID_x,AGE_ID,AGE_NUM,SEX_CODE,RACE_ID,ETHNICITY_ID,RESIDENT_CODE,UNDER_18_DISPOSITION_CODE,CLEARANCE_IND,AGE_RANGE_LOW_NUM,AGE_RANGE_HIGH_NUM,ARREST_TYPE_CODE,ARREST_TYPE_NAME,OFFENSE_CODE,OFFENSE_NAME,CRIME_AGAINST,CT_FLAG,HC_FLAG,HC_CODE,OFFENSE_CATEGORY_NAME,OFFENSE_GROUP,DATA_YEAR_y,OFFENSE_ID,OFFENSE_TYPE_ID_y,ATTEMPT_COMPLETE_FLAG,LOCATION_ID,NUM_PREMISES_ENTERED,METHOD_ENTRY_CODE
0,2019,33189213,111310423,1,2019-03-01,winter,2,N,16,5,13,F,2,2,R,R,NaN,13,0,S,Summoned / Cited,35A,Drug/Narcotic Violations,Society,N,Y,NaN,Drug/Narcotic Offenses,A,2019,136132003,16,C,41,NaN,NaN
1,2019,33182447,111310447,1,2019-03-05,winter,2,N,16,5,29,F,1,2,R,NaN,NaN,29,0,S,Summoned / Cited,35A,Drug/Narcotic Violations,Society,N,Y,NaN,Drug/Narcotic Offenses,A,2019,136125409,35,C,13,NaN,NaN
2,2019,33182447,111310447,1,2019-03-05,winter,2,N,16,5,29,F,1,2,R,NaN,NaN,29,0,S,Summoned / Cited,35A,Drug/Narcotic Violations,Society,N,Y,NaN,Drug/Narcotic Offenses,A,2019,136132061,16,C,13,NaN,NaN
3,2019,33189271,111310549,2,2019-03-18,winter,2,N,16,5,30,M,2,2,R,NaN,NaN,30,0,S,Summoned / Cited,35A,Drug/Narcotic Violations,Society,N,Y,NaN,Drug/Narcotic Offenses,A,2019,136130811,16,C,13,NaN,NaN
4,2019,33182481,111310549,1,2019-03-18,winter,1,N,16,5,28,F,2,2,R,NaN,NaN,28,0,O,On View,35A,Drug/Narcotic Violations,Society,N,Y,NaN,Drug/Narcotic Offenses,A,2019,136130811,16,C,13,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5431,2019,37098240,121201572,1,2019-12-19,autumn,3,N,43,5,51,M,1,2,R,NaN,NaN,51,0,T,Taken INTO Custody,11B,Sodomy,Person,N,Y,2.0,Sex Offenses,A,2019,146744765,8,C,20,NaN,NaN
5432,2019,37098240,121201572,1,2019-12-19,autumn,3,N,43,5,51,M,1,2,R,NaN,NaN,51,0,T,Taken INTO Custody,11B,Sodomy,Person,N,Y,2.0,Sex Offenses,A,2019,146744764,43,C,20,NaN,NaN
5433,2019,34055873,113383897,1,2019-01-22,winter,3,N,38,5,25,M,1,1,R,NaN,NaN,25,0,T,Taken INTO Custody,09B,Negligent Manslaughter,Person,N,Y,NaN,Homicide Offenses,A,2019,138485498,38,C,13,NaN,NaN
5434,2019,37604623,122506312,1,2020-01-10,winter,3,N,7,5,26,M,1,1,R,NaN,NaN,26,0,T,Taken INTO Custody,23G,Theft of Motor Vehicle Parts or Accessories,Property,N,N,NaN,Larceny/Theft Offenses,A,2019,148246444,7,C,18,NaN,NaN


In [142]:
# Merge joined_df3 and location_df on LOCATION_ID.
joined_df4 = joined_df3.merge(location_df, on ="LOCATION_ID")
joined_df4

,DATA_YEAR_x,ARRESTEE_ID,INCIDENT_ID,ARRESTEE_SEQ_NUM,ARREST_DATE,SEASON,ARREST_TYPE_ID,MULTIPLE_INDICATOR,OFFENSE_TYPE_ID_x,AGE_ID,AGE_NUM,SEX_CODE,RACE_ID,ETHNICITY_ID,RESIDENT_CODE,UNDER_18_DISPOSITION_CODE,CLEARANCE_IND,AGE_RANGE_LOW_NUM,AGE_RANGE_HIGH_NUM,ARREST_TYPE_CODE,ARREST_TYPE_NAME,OFFENSE_CODE,OFFENSE_NAME,CRIME_AGAINST,CT_FLAG,HC_FLAG,HC_CODE,OFFENSE_CATEGORY_NAME,OFFENSE_GROUP,DATA_YEAR_y,OFFENSE_ID,OFFENSE_TYPE_ID_y,ATTEMPT_COMPLETE_FLAG,LOCATION_ID,NUM_PREMISES_ENTERED,METHOD_ENTRY_CODE,LOCATION_NAME
0,2019,33189213,111310423,1,2019-03-01,winter,2,N,16,5,13,F,2,2,R,R,NaN,13,0,S,Summoned / Cited,35A,Drug/Narcotic Violations,Society,N,Y,NaN,Drug/Narcotic Offenses,A,2019,136132003,16,C,41,NaN,NaN,School-Elementary/Secondary
1,2019,33184419,111310437,1,2019-03-04,winter,2,N,16,5,15,F,1,1,R,H,NaN,15,0,S,Summoned / Cited,35A,Drug/Narcotic Violations,Society,N,Y,NaN,Drug/Narcotic Offenses,A,2019,136125400,16,C,41,NaN,NaN,School-Elementary/Secondary
2,2019,33189238,111310463,1,2019-03-08,winter,2,N,16,5,15,M,1,2,R,H,NaN,15,0,S,Summoned / Cited,35A,Drug/Narcotic Violations,Society,N,Y,NaN,Drug/Narcotic Offenses,A,2019,136132091,16,C,41,NaN,NaN,School-Elementary/Secondary
3,2019,33182460,111317968,1,2019-03-08,winter,2,N,16,5,14,F,2,2,R,R,NaN,14,0,S,Summoned / Cited,35A,Drug/Narcotic Violations,Society,N,Y,NaN,Drug/Narcotic Offenses,A,2019,136132099,16,C,41,NaN,NaN,School-Elementary/Secondary
4,2019,33189275,111310567,1,2019-03-20,spring,2,N,16,5,14,M,1,1,R,H,NaN,14,0,S,Summoned / Cited,35A,Drug/Narcotic Violations,Society,N,Y,NaN,Drug/Narcotic Offenses,A,2019,136132224,16,C,41,NaN,NaN,School-Elementary/Secondary
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5431,2019,34055863,113389144,1,2019-02-11,winter,1,M,49,5,36,M,2,2,R,NaN,NaN,36,0,O,On View,220,Burglary/Breaking & Entering,Property,Y,Y,5.0,Burglary/Breaking & Entering,A,2019,138494656,5,C,35,NaN,NaN,Gambling Facility/Casino/Race Track
5432,2019,34059542,113385084,1,2019-05-08,spring,3,N,49,5,36,M,2,2,R,NaN,NaN,36,0,T,Taken INTO Custody,220,Burglary/Breaking & Entering,Property,Y,Y,5.0,Burglary/Breaking & Entering,A,2019,138500445,5,C,35,NaN,NaN,Gambling Facility/Casino/Race Track
5433,2019,34059542,113385084,1,2019-05-08,spring,3,N,49,5,36,M,2,2,R,NaN,NaN,36,0,T,Taken INTO Custody,220,Burglary/Breaking & Entering,Property,Y,Y,5.0,Burglary/Breaking & Entering,A,2019,138500444,49,C,35,NaN,F,Gambling Facility/Casino/Race Track
5434,2019,34059558,113397130,1,2019-05-08,spring,3,N,49,5,36,M,2,2,R,NaN,NaN,36,0,T,Taken INTO Custody,220,Burglary/Breaking & Entering,Property,Y,Y,5.0,Burglary/Breaking & Entering,A,2019,138500461,49,C,35,NaN,F,Gambling Facility/Casino/Race Track


In [145]:
# Drop duplicate columns
IL_crime_joined = joined_df4.drop(['DATA_YEAR_x','OFFENSE_TYPE_ID_x','DATA_YEAR_y','OFFENSE_TYPE_ID_y', 'ARREST_DATE'], axis = 1)
IL_crime_joined

,ARRESTEE_ID,INCIDENT_ID,ARRESTEE_SEQ_NUM,SEASON,ARREST_TYPE_ID,MULTIPLE_INDICATOR,AGE_ID,AGE_NUM,SEX_CODE,RACE_ID,ETHNICITY_ID,RESIDENT_CODE,UNDER_18_DISPOSITION_CODE,CLEARANCE_IND,AGE_RANGE_LOW_NUM,AGE_RANGE_HIGH_NUM,ARREST_TYPE_CODE,ARREST_TYPE_NAME,OFFENSE_CODE,OFFENSE_NAME,CRIME_AGAINST,CT_FLAG,HC_FLAG,HC_CODE,OFFENSE_CATEGORY_NAME,OFFENSE_GROUP,OFFENSE_ID,ATTEMPT_COMPLETE_FLAG,LOCATION_ID,NUM_PREMISES_ENTERED,METHOD_ENTRY_CODE,LOCATION_NAME
0,33189213,111310423,1,winter,2,N,5,13,F,2,2,R,R,NaN,13,0,S,Summoned / Cited,35A,Drug/Narcotic Violations,Society,N,Y,NaN,Drug/Narcotic Offenses,A,136132003,C,41,NaN,NaN,School-Elementary/Secondary
1,33184419,111310437,1,winter,2,N,5,15,F,1,1,R,H,NaN,15,0,S,Summoned / Cited,35A,Drug/Narcotic Violations,Society,N,Y,NaN,Drug/Narcotic Offenses,A,136125400,C,41,NaN,NaN,School-Elementary/Secondary
2,33189238,111310463,1,winter,2,N,5,15,M,1,2,R,H,NaN,15,0,S,Summoned / Cited,35A,Drug/Narcotic Violations,Society,N,Y,NaN,Drug/Narcotic Offenses,A,136132091,C,41,NaN,NaN,School-Elementary/Secondary
3,33182460,111317968,1,winter,2,N,5,14,F,2,2,R,R,NaN,14,0,S,Summoned / Cited,35A,Drug/Narcotic Violations,Society,N,Y,NaN,Drug/Narcotic Offenses,A,136132099,C,41,NaN,NaN,School-Elementary/Secondary
4,33189275,111310567,1,spring,2,N,5,14,M,1,1,R,H,NaN,14,0,S,Summoned / Cited,35A,Drug/Narcotic Violations,Society,N,Y,NaN,Drug/Narcotic Offenses,A,136132224,C,41,NaN,NaN,School-Elementary/Secondary
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5431,34055863,113389144,1,winter,1,M,5,36,M,2,2,R,NaN,NaN,36,0,O,On View,220,Burglary/Breaking & Entering,Property,Y,Y,5.0,Burglary/Breaking & Entering,A,138494656,C,35,NaN,NaN,Gambling Facility/Casino/Race Track
5432,34059542,113385084,1,spring,3,N,5,36,M,2,2,R,NaN,NaN,36,0,T,Taken INTO Custody,220,Burglary/Breaking & Entering,Property,Y,Y,5.0,Burglary/Breaking & Entering,A,138500445,C,35,NaN,NaN,Gambling Facility/Casino/Race Track
5433,34059542,113385084,1,spring,3,N,5,36,M,2,2,R,NaN,NaN,36,0,T,Taken INTO Custody,220,Burglary/Breaking & Entering,Property,Y,Y,5.0,Burglary/Breaking & Entering,A,138500444,C,35,NaN,F,Gambling Facility/Casino/Race Track
5434,34059558,113397130,1,spring,3,N,5,36,M,2,2,R,NaN,NaN,36,0,T,Taken INTO Custody,220,Burglary/Breaking & Entering,Property,Y,Y,5.0,Burglary/Breaking & Entering,A,138500461,C,35,NaN,F,Gambling Facility/Casino/Race Track


In [146]:
# Write IL_crime DataFrame to a csv file.
IL_crime_joined.to_csv('mock_df.csv', index=False)

In [340]:
#seasons_encoded = pd.get_dummies(get_season, columns=["season_data"])
#seasons_encoded

In [341]:
# Define the features set
X = df.drop(["ARREST_DATE", "CLEARANCE_IND", "season"], axis=1).fillna(0)

# Display the features set
X

y = df["season"]

NameError: name 'df' is not defined

In [ ]:
#X["CLEARANCE_IND"].value_counts()


In [ ]:
# # Define the target set
# y = seasons.values
# #y = df["ARREST_DATE"].values
# y[:5]

In [ ]:
X

In [ ]:
#list(X[col].values)

In [ ]:
col_to_be_encoded = ["MULTIPLE_INDICATOR", "SEX_CODE", "RESIDENT_CODE", "UNDER_18_DISPOSITION_CODE"]
for col in col_to_be_encoded:
    le = preprocessing.LabelEncoder()
    X[col] = le.fit_transform(list(X[col].values))
    pickle.dump(le, open(f"{col}.pkl","wb"))

X

In [ ]:
le = preprocessing.LabelEncoder()
y_encoded = le.fit_transform(list(y.values))
pickle.dump(le, open(f"season.pkl","wb"))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, random_state=78)

In [ ]:
# Determine the shape of our training and testing sets.
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
list(y_train)

In [ ]:
clf = RandomForestClassifier()
clf.fit(X, y_encoded)

In [ ]:
predictions = clf.predict(X_test)

In [ ]:
# Calculating the accuracy score.
acc_score = accuracy_score(y_test, predictions)
acc_score

In [ ]:
joblib.dump(clf, "trained_random_forest.joblib")

In [ ]:
import pandas as pd
offense_type_df = pd.read_csv("IL/NIBRS_OFFENSE_TYPE.csv")
offense_type_df

In [ ]:
offenses_df = 

In [ ]:
months = {"JAN" : "January", "FEB" : "February", "MAR" : "March", "APR" : "April", "MAY" : "May", "JUN" : "June", "JUL" : "July", "AUG" : "August", "SEP" : "September", "OCT" : "October", "NOV" : "November", "DEC" : "December"}
months["JAN"]

In [ ]:
months_1 = {"JAN", "FEB", "MAR", "APR", "MAY", "JUN", "JUL", "AUG", "SEP", "OCT", "NOV", "DEC"}
for i in months_1:
    datetime.datetime.strptime(i,'%b').strftime('%B')
print(months_1)

In [ ]:
for i in df['ARREST_DATE']:
    datetime.datetime.strptime('i','%b').strftime('%B')

In [ ]:
april = datetime.datetime.strptime('APR','%b').strftime('%B')
april

In [ ]:
from datetime import date, datetime

Y = 2019 

seasons = [('winter', (date(Y,  1,  1),  date(Y,  3, 20))),
           ('spring', (date(Y,  3, 21),  date(Y,  6, 20))),
           ('summer', (date(Y,  6, 21),  date(Y,  9, 22))),
           ('autumn', (date(Y,  9, 23),  date(Y, 12, 20))),
           ('winter', (date(Y, 12, 21),  date(Y, 12, 31)))]

def get_season(now):
    if isinstance(now, datetime):
        now = now.date()
    now = now.replace(year=Y)
    return next(season for season, (start, end) in seasons
                if start <= now <= end)

# print(get_season(date.today()))
#for i in seasons:
#    print(i);
    

In [ ]:
import datetime
d = datetime.date(2019, 7 , 24)
print(d)

In [ ]:
today = datetime.date.today()
print(today)

In [ ]:
birthday = datetime.date(1993, 8, 1)
print(birthday)

In [ ]:
days_since_birth = (today - birthday).days
print(days_since_birth)

In [ ]:
tdelta = datetime.timedelta(days=10)
print(today - tdelta)

In [ ]:
print(today.month)

In [ ]:
print(today.day)

In [ ]:
print(today.weekday())

In [ ]:
print(datetime.time(7, 2, 20, 15))

In [ ]:
# Add 10 hours to current day 
hour_delta = datetime.timedelta(hours=10)
print(datetime.datetime.now() + hour_delta)

In [ ]:
datetime_today = datetime.datetime.now(tz=pytz.UTC)
datetime_pacific = datetime_today.astimezone(pytz.timezone('US/Pacific'))
print(datetime_pacific)
for tz in pytz.all_timezones:
    print(tz)

In [ ]:
# string formatting with dates
# 2019-03-09 -> March 3, 2019
print(datetime_pacific.strftime('%B %d %Y'))